# Dataloader ECL

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pandas as pd
import numpy as np


Le ECL dataset est un dataset sur la consommation d'electricité de 321 clients toutes les heures entre 2012 et 2014.   
Les colonnes représentent les 321 clients et il y a 26 304 lignes. 
 

In [109]:
ecl_data = pd.read_csv('electricity.csv', header=None).to_numpy()

# Diviser les données en ensembles de train, test et validation
train_data = ecl_data[:18317]
test_data = ecl_data[18317:18317+2633]
val_data = ecl_data[18317+2633:]


## DataLoader

In [110]:
class MonDataLoader(Dataset):
  def __init__(self, data, lookback_size, lookforward_size):
    self.lookback_size = lookback_size
    self.lookforward_size = lookforward_size
    self.data = []
    for i in range(0, 1 + len(data) - (self.lookback_size+ self.lookforward_size), self.lookback_size):
      seq_x = data[i:i+self.lookback_size, :]
      seq_y = data[i+self.lookback_size:i+self.lookback_size+self.lookforward_size, :]
      self.data.append((seq_x, seq_y))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    seq_x, seq_y = self.data[idx]
    return torch.tensor(seq_x), torch.tensor(seq_y)

#### dataloader

In [111]:
# Diviser les données en ensembles de train, test et validation
train_data = ecl_data[:18317]
test_data = ecl_data[18317:18317+2633]
val_data = ecl_data[18317+2633:]

In [117]:
batch_size = 32

train_dataset = MonDataLoader(ecl_data[:18317], 96, 96)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

eval_dataset = MonDataLoader(ecl_data[18317+2633:], 96, 96)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

test_dataset = MonDataLoader(ecl_data[18317+2633:], 96, 96)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [121]:
for x,y in test_loader: 
    print(x.shape)
    print(y.shape)


torch.Size([32, 96, 321])
torch.Size([32, 96, 321])
torch.Size([22, 96, 321])
torch.Size([22, 96, 321])


## Transformer

In [2]:
class SAttention(nn.Module):
  def __init__(self, k, heads, mask=False):
    """

    :param k: taille de l'embeding
    :param heads: nombre de heads
    :param mask: attention sur toute la seq(False) ou les elts precs seulement
    """
    
    super(SAttention, self).__init__()

    self.k = k
    self.heads = heads
    self.mask = mask
    
    self.tokeys = nn.Linear(k, k, bias=False)
    self.toqueries = nn.Linear(k, k, bias=False)
    self.tovalues = nn.Linear(k, k, bias=False)

    self.unifyHeads = nn.Linear(k,k)
  
  def forward(self, x):
    
    b, t, k = x.size()
    h = self.heads
    assert k == self.k, f'Taille des embeddings ({k}) doit correspond a celui du init ({self.k})'

    keys = self.tokeys(x)
    queries = self.toqueries(x)
    values = self.tovalues(x)
    
    # Taille de chaque head
    s = k // h
    
    # batch x longueur seq x nb heads x taille head
    keys = keys.view(b, t, h, s)
    queries = queries.view(b, t, h, s)
    values = values.view(b, t, h, s)
    
    # batch et head cote à cote
    keys = keys.transpose(1, 2).contiguous().view(b * h, t, s)
    queries = queries.transpose(1, 2).contiguous().view(b * h, t, s)
    values = values.transpose(1, 2).contiguous().view(b * h, t, s)

    dot = torch.bmm(queries, keys.transpose(1, 2)) # -> (b*h, t, t)
    dot = dot / (k ** (1/2))
    
    if self.mask:
      indices = torch.triu_indices(t, t, offset=1)
      dot[:, indices[0], indices[1]] = float('-inf')

    dot = F.softmax(dot, dim=2)
    
    out = torch.bmm(dot, values).view(b, h, t, s)

    # rearrangement
    out = out.transpose(1, 2).contiguous().view(b, t, h*s)

    # unification avec MLP
    out = self.unifyHeads(out) # -> (b, t, k)

    return out


In [ ]:
"""
def forward_einsum(self, x):
    b, t, e = x.size()
    h = self.heads

    keys    = self.tokeys(x).view(b, t, h, e)
    queries = self.toqueries(x).view(b, t, h, e)
    values  = self.tovalues(x).view(b, t, h, e)

    dot = torch.einsum('bthe,bihe->bhti', queries, keys) / math.sqrt(e)
    dot = F.softmax(dot, dim=-1)

    out = torch.einsum('bhtd,bdhe->bthe', dot, values)

    # we can move reshape of weights to init; I left it here just to compare with the original implementation
    out = torch.einsum('bthe,khe->btk', out, self.unifyheads.weight.view(e,h,e)) 
    return out + self.unifyheads.bias
"""

In [3]:
class TBlock(nn.Module):
  def __init__(self, k, heads):
    super(TBlock, self).__init__()
    
    self.attention = SAttention(k, heads)
    self.norm1 = nn.LayerNorm(k)
    self.norm2 = nn.LayerNorm(k)

    self.fedForward = nn.Sequential(
      nn.Linear(k, 4*k),
      nn.ReLU(),
      nn.Linear(4*k, k)
    )
  
  def forward(self, x):
    attention = self.attention(x)
    x = x + attention
    x = self.norm1(x)
    
    fedForward = self.fedForward(x)
    x = x + fedForward
    out = self.norm2(x)

    return out

In [ ]:
class Transformer(nn.Module):
  def __init__(self, k, heads, nb_TBlocks, in_seq_len, out_seq_len):
    # in_seq_len pas interessant ici
    
    self.k = k
    self.heads = heads
    self.nb_TBlocks = nb_TBlocks
    self.in_seq_len = in_seq_len
    self.out_seq_len = out_seq_len
    
    self.pos_emb = nn.Embedding(in_seq_len, k)

    tblocks = [TBlock(k=k, heads=heads) for _ in range(nb_TBlocks)]
    self.tblocks = nn.Sequential(*tblocks)
    
    # projection
    self.projection = nn.Sequential(
      nn.Linear(k, 4*k),
      nn.ReLU(),
      nn.Linear(k, 8*k),
      nn.ReLU(),
      nn.Linear(8*k, 16*k),
      nn.ReLU(),
      nn.Linear(16*k, out_seq_len*k)
    )
  
  def forward(self, x):
    """

    Args:
      x (b, t, k): entree batch de seq(taille in_seq_len), chaque elt de seq de taille (k)
    """
    b, t, k = x.size()
    assert t == self.in_seq_len, f'Taille de la sequence ({t}) doit correspondre à in_seq_len({self.in_seq_len})'
    assert k == self.k, f'Taille emb ({k}) doit correspondre à k({self.k})'
    

    pos = torch.arange(t)
    pos = self.pos_emb(pos)[None, :, :].expand(b, t, k)

    x = x + pos
    x = self.tblocks(x)

    # prediction  (b, out_seq_len, k)
    out = self.projection(x)
    out = out.view(self.out_seq_len, self.k)
    
    return out
    